In [ ]:

# from google.colab import files 
# uploaded = files.upload()
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import models, layers
import numpy as np

In [ ]:
BATCH_SIZE= 32
IMAGE_SIZE= 256
CHANNELS=3
EPOCHS=50
input_shape= (BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory("/content/gdrive/MyDrive/flowers",shuffle
                                                              =True,image_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                              batch_size=BATCH_SIZE)

In [ ]:
class_names= dataset.class_names
class_names

In [ ]:
plt.figure(figsize=(10,10))
for image_batch,labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())
    for i in range(11):
        ax= plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

In [ ]:
def get_dataset_part(ds,train_split=0.8,val_split=0.1,test_split=0.1,shufffle=True):
    assert(train_split+val_split+test_split)==1
    ds_size= len(ds)
    train_size= int(train_split*ds_size)
    val_size= int(val_split*ds_size)
    
    train_ds= ds.take(train_size)
    val_ds= ds.skip(train_size).take(val_size)
    test_ds= ds.skip(train_size).skip(val_size)
    
    return train_ds,val_ds,test_ds

In [ ]:

train_size=0.8
len(dataset)*train_size

In [ ]:
train_ds,val_ds,test_ds= get_dataset_part(dataset)

In [ ]:
len(test_ds)

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1.0/255),
])

In [ ]:

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
n_classes=5
model = models.Sequential([resize_and_rescale,
                  data_augmentation,
                  layers.Conv2D(32,kernel_size= (3,3),activation='relu',input_shape=input_shape),
                 layers.MaxPooling2D((2,2)),
                   layers.Conv2D(64,(3,3),activation='relu'),
                 layers.MaxPooling2D((2,2)),
                   layers.Conv2D(64,(3,3),activation='relu'),
                 layers.MaxPooling2D((2,2)),
                   layers.Conv2D(64,kernel_size= (3,3),activation='relu'),
                 layers.MaxPooling2D((2,2)),
                   layers.Flatten(),
                   layers.Dense(64,activation='relu'),
                   layers.Dense(n_classes,activation='softmax')
                 ])
model.build(input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics=['accuracy']
             )

In [ ]:
history = model.fit(train_ds,batch_size=BATCH_SIZE,validation_data=val_ds,verbose=1,
                   epochs=15)

In [ ]:
model.evaluate(test_ds)

In [ ]:
history.history.keys()

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
for image_batch,labels_batch in dataset.take(1):
    img= image_batch[0].numpy().astype("uint8")
    label=labels_batch[0].numpy()
    plt.imshow(img)
    print("Actual Label : ",class_names[labels_batch[0].numpy()])
    
    batch_prediction= model.predict(image_batch)
    print("Predicted Label : ",class_names[np.argmax(batch_prediction[0])])

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:

model.save("./flower_model.h5")